# Inception-ResNet-V2 keras implementation using CIFAR10

In [1]:
import numpy as np
import keras
from keras import layers
import keras.backend as K
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.callbacks import CSVLogger, EarlyStopping
from keras.initializers import glorot_uniform
import cv2

K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [31]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = np.array([cv2.resize(img, (128,128)) for img in X_train[:10000,:,:,:]])
X_test = np.array([cv2.resize(img, (128,128)) for img in X_test[:1000,:,:,:]])

n_classes = 10

Y_train = np_utils.to_categorical(Y_train[:10000,:], n_classes)
Y_test = np_utils.to_categorical(Y_test[:1000,:], n_classes)

mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [18]:
def inception_resnet_A(input_x):
    conv1_1x1 = layers.Conv2D(32, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    
    conv2_1x1 = layers.Conv2D(32, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    conv2_3x3 = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv2_1x1)
    
    conv3_1x1 = layers.Conv2D(32, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    conv3_3x3 = layers.Conv2D(48, kernel_size=(3,3), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv3_1x1)
    conv3_3x3 = layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv3_1x1)
    
    merged = layers.Concatenate(axis = -1)([conv1_1x1, conv2_3x3, conv3_3x3])
    
    output1 = layers.Conv2D(384, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(merged)
    
    output = layers.Add()([input_x, output1])
    
    return output

In [19]:
def inception_resnet_B(input_x):
    conv1_1x1 = layers.Conv2D(192, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    
    conv2_1x1 = layers.Conv2D(128, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    conv2_1x7 = layers.Conv2D(160, kernel_size=(1,7), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv2_1x1)
    conv2_7x1 = layers.Conv2D(192, kernel_size=(7,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv2_1x7)
    
    merged = layers.Concatenate(axis=-1)([conv1_1x1, conv2_7x1])
    
    output1 = layers.Conv2D(1152, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(merged)
    
    output = layers.Add()([input_x, output1])
    
    return output
    

In [20]:
def inception_resnet_C(input_x):
    conv1_1x1 = layers.Conv2D(192, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    
    conv2_1x1 = layers.Conv2D(192, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(input_x)
    conv2_1x3 = layers.Conv2D(224, kernel_size=(1,3), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv2_1x1)
    conv2_3x1 = layers.Conv2D(256, kernel_size=(3,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(conv2_1x3)
    
    merged = layers.Concatenate(axis=-1)([conv1_1x1, conv2_3x1])
    
    output1 = layers.Conv2D(2144, kernel_size=(1,1), activation='relu', padding="same", strides=(1,1), kernel_initializer=glorot_uniform())(merged)
    
    output = layers.Add()([input_x, output1])
    
    return output

In [21]:
def reduction_A(input_x):
    max_pool1_3x3 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(input_x)
    
    conv2_3x3 = layers.Conv2D(384, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(input_x)
    
    conv3_1x1 = layers.Conv2D(256, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(input_x)
    conv3_3x3_a = layers.Conv2D(256, kernel_size=(3,3), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(conv3_1x1)
    conv3_3x3_b = layers.Conv2D(384, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(conv3_3x3_a)
    
    output= layers.Concatenate(axis=-1)([max_pool1_3x3, conv2_3x3, conv3_3x3_b])

    return output
    

In [22]:
def reduction_B(input_x):
    max_pool1_3x3 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(input_x)
    
    conv2_1x1 = layers.Conv2D(256, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(input_x)
    conv2_3x3 = layers.Conv2D(384, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(conv2_1x1)
    
    conv3_1x1 = layers.Conv2D(256, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(input_x)
    conv3_3x3 = layers.Conv2D(288, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(conv3_1x1)
    
    conv4_1x1 = layers.Conv2D(256, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(input_x)
    conv4_3x3_a = layers.Conv2D(288, kernel_size=(3,3), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(conv4_1x1)
    conv4_3x3_b = layers.Conv2D(320, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(conv4_3x3_a)
    
    output= layers.Concatenate(axis=-1)([max_pool1_3x3, conv2_3x3, conv3_3x3, conv4_3x3_b])
    
    return output
    
    
    
    

In [27]:
def inception_resnet_v2(input_shape, n_classes):
    input_x = layers.Input(input_shape)
    
    X = input_x
    
    X = layers.Conv2D(32, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(X)
    X = layers.Conv2D(32, kernel_size=(3,3), activation="relu", strides=(1,1), padding="valid", kernel_initializer=glorot_uniform())(X)
    X = layers.Conv2D(64, kernel_size=(3,3), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(X)
    
    X1 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    X2 = layers.Conv2D(96, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(X)
    
    X = layers.Concatenate(axis=-1)([X1, X2])
    
    X1 = layers.Conv2D(64, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(X)
    X1 = layers.Conv2D(96, kernel_size=(3,3), activation="relu", strides=(1,1), padding="valid", kernel_initializer=glorot_uniform())(X1)
    X2 = layers.Conv2D(64, kernel_size=(1,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(X)
    X2 = layers.Conv2D(64, kernel_size=(7,1), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(X2)
    X2 = layers.Conv2D(64, kernel_size=(1,7), activation="relu", strides=(1,1), padding="same", kernel_initializer=glorot_uniform())(X2)
    X2 = layers.Conv2D(96, kernel_size=(3,3), activation="relu", strides=(1,1), padding="valid", kernel_initializer=glorot_uniform())(X2)
    
    X = layers.Concatenate(axis=-1)([X1, X2])
    
    X1 = layers.Conv2D(192, kernel_size=(3,3), activation="relu", strides=(2,2), padding="valid", kernel_initializer=glorot_uniform())(X)
    X2 = layers.MaxPool2D(pool_size=(3,3), strides=(2,2), padding="valid")(X)
    
    X = layers.Concatenate(axis=-1)([X1, X2])
    
    #print(X._keras_shape)
    
    for _ in range(5):
        X = inception_resnet_A(X)
    
    #print(X._keras_shape)
    
    X = reduction_A(X)
    
    #print(X._keras_shape)
    
    for _ in range(10):
        X = inception_resnet_B(X)
    
    #print(X._keras_shape)
    
    X = reduction_B(X)
    
    #print(X._keras_shape)
    
    X = inception_resnet_C(X)
    
    #print(X._keras_shape)
    
    #pool_size=(8,8) in orignal implementation
    X = layers.AveragePooling2D(pool_size=(2,2), strides=(1,1), padding="valid")(X)
    
    X = layers.Flatten()(X)
    
    X = layers.Dropout(0.2)(X)
    
    X = layers.Dense(n_classes, activation="softmax")(X)
    
    model = keras.Model(inputs=input_x, outputs=X)
    
    return model


    
    

In [32]:
inception_resnet_v2_model = inception_resnet_v2(input_shape=(128,128,3),n_classes=10)

In [33]:
inception_resnet_v2_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
inception_resnet_v2_model.fit(X_train, Y_train, epochs = 1, batch_size = 128)

Epoch 1/1
10000/10000 [==============================] - 1083s 108ms/step - loss: 14.3848 - acc: 0.0975
